In [1]:
import warnings
warnings.filterwarnings('ignore')

Sentence_Transfomer Model Architecture
Transformer Models: The library utilizes transformer models like BERT, RoBERTa, DistilBERT, etc., which are pre-trained on large corpora and can be fine-tuned for specific tasks.
Pooling Strategies: After processing input text through a transformer, a pooling layer aggregates token embeddings to form a single embedding for the entire sentence or text. Common strategies include mean, max, and CLS token pooling.
Applications
Semantic Textual Similarity: By embedding sentences into high-dimensional vectors, the library can be used to compare and compute similarities between texts, useful for tasks such as duplicate detection or information retrieval.
Clustering and Classification: Sentence embeddings can be used for text classification or clustering, grouping similar texts together based on their semantic content.



**Using all-MiniLM-L6-v2 sentence-transformers model that maps sentences to a 384 dimensional dense vector space.

In [2]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
import os
import time
import torch
from tqdm.auto import tqdm
from dotenv import load_dotenv

In [3]:
load_dotenv()
dataset = load_dataset('quora', split='train[240000:290000]')

In [7]:
dataset[:1]

{'questions': [{'id': [207550, 351729],
   'text': ['What is the truth of life?', "What's the evil truth of life?"]}],
 'is_duplicate': [False]}

In [8]:
questions = []
for record in dataset['questions']:
    questions.extend(record['text'])
question = list(set(questions))
print('\n'.join(questions[:10]))
print('-' * 50)
print(f'Number of questions: {len(questions)}')

What is the truth of life?
What's the evil truth of life?
Which is the best smartphone under 20K in India?
Which is the best smartphone with in 20k in India?
Steps taken by Canadian government to improve literacy rate?
Can I send homemade herbal hair oil from India to US via postal or private courier services?
What is a good way to lose 30 pounds in 2 months?
What can I do to lose 30 pounds in 2 months?
Which of the following most accurately describes the translation of the graph y = (x+3)^2 -2 to the graph of y = (x -2)^2 +2?
How do you graph x + 2y = -2?
--------------------------------------------------
Number of questions: 100000


In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print('No cuda.')
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

No cuda.


In [13]:
q="Which city has most engineers in USA?"
q_embed=model.encode(q)
q_embed.shape

(384,)

In [16]:
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

For Below code:

    create_index is a method that initializes a new vector index. 
    
    Parameters of create_index
        name=INDEX_NAME: This specifies the name of the index. This name is used to reference the index in subsequent operations, such as adding or querying vectors.

        dimension=model.get_sentence_embedding_dimension(): This sets the dimensionality of the vectors that will be stored in the index. 
        
        metric='cosine': This defines the metric used to measure similarity between vectors in the index. The cosine similarity metric measures the cosine of the angle between two vectors, providing a value that indicates how similar they are in orientation in the vector space, regardless of their magnitude. 
     

        spec=ServerlessSpec(cloud='aws', region='us-west-2'): This parameter specifies the serverless configuration for the index:

        ServerlessSpec: Configuration structure in the Pinecone library that defines how the index should be deployed.
            cloud='aws': hosted on Amazon Web Services (AWS).
            region='us-west-2': AWS region where the index will be physically created. 

In [20]:
pc=Pinecone(api_key=PINECONE_API_KEY)
INDEX_NAME="semanticsearchpinecone1"
if INDEX_NAME in [index.name for index in pc.list_indexes()]:
    pc.delete_index(INDEX_NAME)
    
print(INDEX_NAME)

semanticsearchpinecone1


In [21]:
pc.create_index(name=INDEX_NAME, 
                dimension=model.get_sentence_embedding_dimension(), 
                metric='cosine', 
                spec=ServerlessSpec(cloud='aws', region='us-west-2'))

In [23]:
index = pc.Index(INDEX_NAME)
print(index)

create embeddings and upsert.....

In [27]:
batch=250
q_vec_limit=10000
questions = question[:q_vec_limit]

import json

for i in tqdm(range(0, len(questions), batch)):
    i_end = min(i+batch, len(questions))
    ids = [str(x) for x in range(i, i_end)]
    metadatas = [{'text': text} for text in questions[i:i_end]]
    print(len(ids), ' - '*10,  metadatas)
    print("-"*50)

  2%|▎         | 1/40 [00:01<00:39,  1.00s/it]

250  -  -  -  -  -  -  -  -  -  -  [{'text': "What is the Secret Service's nickname for Trump?"}, {'text': 'Should we really worry about climate change?'}, {'text': "What does the word 'somberi' mean in Tamil?"}, {'text': 'Does 18 month are enough to crack IIT JEE?'}, {'text': 'Do teenage guys fall in love?'}, {'text': 'What did the German soldiers of WWII think of British, US, Canadian, and Soviet soldiers?'}, {'text': 'What is the chemical formula for zinc? How is this determined?'}, {'text': "I'm a 5 letter word. I am normally below you. If you remove my 1st letter, you'll find me above you. If you remove my 1st & 2nd letters, you can't see me. What am I?"}, {'text': 'Survivor TV show: when going to tribal council, how far do the players walk? They often leave during daylight and get there in the dark?'}, {'text': 'Is Google Glass a flop?'}, {'text': "Who are the likely members of Donald Trump's cabinet?"}, {'text': 'What is an OTG pen drive?'}, {'text': "Should I get a fully paid e

  5%|▌         | 2/40 [00:01<00:36,  1.03it/s]

250  -  -  -  -  -  -  -  -  -  -  [{'text': 'How would world history change if the Earth had an obliquity/tilt of 0 degrees?'}, {'text': 'Were playstation 1 games written mainly in C or MIPS assembly?'}, {'text': 'Dating and Relationships: If a girl likes me, but she blocked me on Facebook, MSN, and refused to reply to my texts, what does it mean?'}, {'text': 'Is there a way of disproving the conviction that reality is merely a construct of our mind and essentially a figment of our imagination?'}, {'text': "Why do people mark questions as ambiguous or needing editing on Quora when they're fine?"}, {'text': 'What are the best reference books for learning Java?'}, {'text': 'What is the fees of luxury and brand management course from SP Jain school of global management?'}, {'text': 'Who will be the next Chief Minister of Maharashtra?'}, {'text': 'Why the "GST\' bill is not getting passed?'}, {'text': "How do you delete an instagram account when you don't remember the password or username

  8%|▊         | 3/40 [00:02<00:31,  1.16it/s]

...

250  -  -  -  -  -  -  -  -  -  -  [{'text': 'How do I create a middle finger symbol with text?'}, {'text': 'What are the types of Rationality offered by Max Weber?'}, {'text': 'How do you reverse a linked list?'}, {'text': 'How was education during the Japanese occupation like in Singapore?'}, {'text': "In India, what are the things that women can do easily, but men can't?"}, {'text': 'How does spilled water damage a laptop?'}, {'text': 'What is your most embarrassing injury?'}, {'text': 'If I forgot my Facebook password, how do I reset it?'}, {'text': 'How can sell my land online?'}, {'text': "What's the difference between performance and efficiency?"}, {'text': 'What is an example of the word "dour" in a sentence?'}, {'text': 'Can I know the basics of hacking?'}, {'text': 'What statistical procedure would be used to determine black money after demonitisation?'}, {'text': 'What are some of the best places to visit in your country?'}, {'text': 'I have a client in the US who is ready t

 98%|█████████▊| 39/40 [00:47<00:01,  1.23s/it]

250  -  -  -  -  -  -  -  -  -  -  [{'text': 'What are the distinct behavioral differences between an introvert and a schizoid?'}, {'text': 'What is global solidarity?'}, {'text': 'What can you change in your life to become an everyday hero?'}, {'text': 'How do I recover a lost password or account for gmail?'}, {'text': 'Which other countries did demonetization?'}, {'text': 'How can you study economics on Quora?'}, {'text': 'Is sex painful during pregnancy?'}, {'text': 'Why do some people believe that the world is flat?'}, {'text': 'How can I get back my original skin colour? Its tanned.'}, {'text': 'Is it ok if a IT professional works on a freelancing work?'}, {'text': 'How do I prepare for the NMAT in 10 days?'}, {'text': "How long would it take to load a 1GB file into RAM if the speed of the drive the file was stored on didn't matter?"}, {'text': 'Can $B creative breakthroughs like Uber happen on demand?'}, {'text': 'What are the problems after upgrading to Windows 10?'}, {'text': '

100%|██████████| 40/40 [00:48<00:00,  1.21s/it]

250  -  -  -  -  -  -  -  -  -  -  [{'text': 'What are some major technical breakthroughs in recent video games?'}, {'text': 'How can I get 100 likes on my Facebook photo?'}, {'text': 'What is the difference between Sin and Crime?'}, {'text': 'What are the sites to download TV series?'}, {'text': 'Will the 2016 US presidential election be close?'}, {'text': 'What is the best SEO company in Delhi, India?'}, {'text': "Moto g4 screen is all black lights light up but can't see anything reset tricks do not work?"}, {'text': 'How can I publish theses on agriculture on the International Library for Thesis?'}, {'text': 'What is best i5 6th or 7th gen?'}, {'text': 'What is the way to overcome the stage fear?'}, {'text': 'What is your horror story at NIT?'}, {'text': "What's the best way to extend battery life on my Android phone?"}, {'text': 'Hypothetical scenario: the US military is forced to choose between giving up all its Apache Gunships or all its Black Hawks. Which do they choose?'}, {'te

insert/upsert to pincode with embeddings and metadata

In [28]:
for i in tqdm(range(0, len(questions), batch)):
    i_end = min(i+batch, len(questions))
    ids = [str(x) for x in range(i, i_end)]
    metadatas = [{'text': text} for text in questions[i:i_end]]
    xc = model.encode(questions[i:i_end])
    records = zip(ids, xc, metadatas)
    index.upsert(vectors=records)

100%|██████████| 40/40 [01:06<00:00,  1.66s/it]


In [29]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 10000}},
 'total_vector_count': 10000}

Below code:

index.query(...): This function call performs the actual search. The index is a vector search index, potentially managed by a library such as Pinecone, Faiss, or Annoy.

top_k=10: The search query returns the top 10 results that are most similar to the provided vector.
vector=embedding: This specifies the query vector.

include_metadata=True: Indicates that the query should return metadata associated with each result. Metadata often includes original text or additional information useful for identifying or understanding the results.

include_values=False: Specifies that the actual vector values of the results should not be included in the response. This is useful for reducing response size if only metadata is required.
    
    > What Does include_values=False Do?
        When you perform a vector search query, the search engine can return various pieces of information about each of the matches found. These pieces of information can include:

            Metadata: This is data associated with each item in the vector database, such as descriptions, titles, or any other contextual information that was stored with the vector. Metadata helps in identifying or understanding what each vector represents in human-readable form.

            Vector Values: These are the actual high-dimensional data points (numerical values) of each vector. Vectors are the core component of how items are stored and searched within the system. They represent the item in a mathematical space.

In [38]:
# small helper function so we can repeat queries later
def query(query):
  embedding = model.encode(query).tolist()
  results = index.query(top_k=10, vector=embedding, include_metadata=True, include_values=False)
  for result in results['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")

In [39]:
query('AM I BATMAN!!!!')

0.51: Who is a better superhero: Batman or Spider-Man?
0.5: Why does Joker in Batman so famous?
0.45: Who would win a fight between wolverine and Batman?
0.45: What fighting style does Batman use in Batman vs Superman?
0.44: How should I prepare myself to be a real-life Joker as in The Dark Knight?
0.43: Is the Joker in the Suicide Squad comics?
0.4: If Batman had gone to Hogwarts, which house would he be in?
0.37: What is the full form of DC in DC comics?
0.37: The Dark Knight Rises (2012 movie): Why didn't Batman kill Bane in this scene when he had a clear shot at him with "the bat"?
0.36: Why does The Joker burn the money in The Dark Knight?
